## Atari Breakout Q-Learning

##### Allow memory growth for tensorflow, Windows work around

In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

##### Verify GPU availability

from tensorflow.python.client import device_lib  
print(device_lib.list_local_devices())  
from keras import backend as K  
print(K.tensorflow_backend._get_available_gpus())  

#### Training

In [2]:
from ai_image_preprocess import preprocess

def step(env, action, state):
    next_frame_1, reward_1, done_1, _ = env.step(action)
    next_frame_2, reward_2, done_2, _ = env.step(action)
    next_state = util.update_state_arr(state, preprocess(next_frame_1), preprocess(next_frame_2))
    return (next_state, reward_1 + reward_2, done_1 or done_2)

In [3]:
import ai_display as display
import ai_util as util
import ai_logger as logger
import time

def train(env, agent, n_episodes, model_name="model", save_interval=100):
    for episode in range(n_episodes):
        
        frame = env.reset()
        state = util.create_state_arr(preprocess(frame))
        score = 0
        
        start_time = time.time()
        for t in range(1000):
            #display.show_state(state[3], env.spec.id, t, score)

            action = agent.act(state)

            next_state, reward, done = step(env, action, state)
            
            agent.remember(state, action, reward, next_state, done)
            
            state = next_state
            score += reward
            
            agent.replay(batch_size=32)
            
            if done: 
                logger.log("Episode {:>5}/{}, #frames {:>3}, score {}, avg_step_time {:.5f}"
                           .format(episode+1, n_episodes, t, score, (time.time() - start_time)/t))
                break
                
        if episode % save_interval == 0:
            agent.save_model(model_name)
        
    agent.save_model(model_name)
        

#### Main

In [ ]:
from ai_agent import Agent
import gym

env = gym.make("BreakoutDeterministic-v4")
n_actions = env.action_space.n
observation_space = env.observation_space.high
state_shape = (105, 80, 4)

agent = Agent(state_shape, n_actions, epsilon=0.6)

model_name = "new-model"
#agent.new_model()
agent.load_model(model_name) 
train(env, agent, n_episodes=10000, model_name=model_name, save_interval=10)
#step_time(env, agent)

total 0.06224, act 0.00066, step 0.00277, remember 0.00023, replay 0.05859
total 0.06289, act 0.00093, step 0.00291, remember 0.00014, replay 0.05891
total 0.06301, act 0.00097, step 0.00298, remember 0.00009, replay 0.05895
total 0.06260, act 0.00091, step 0.00281, remember 0.00015, replay 0.05873
total 0.06242, act 0.00077, step 0.00287, remember 0.00016, replay 0.05862
total 0.06440, act 0.00080, step 0.00295, remember 0.00008, replay 0.06057
total 0.06208, act 0.00079, step 0.00293, remember 0.00014, replay 0.05822
total 0.06205, act 0.00089, step 0.00291, remember 0.00012, replay 0.05813
total 0.06213, act 0.00073, step 0.00286, remember 0.00011, replay 0.05843
total 0.06089, act 0.00077, step 0.00282, remember 0.00006, replay 0.05724
